In [ ]:
!!pip install azure-eventhub

In [ ]:
from azure.eventhub import EventHubProducerClient, EventData
import pandas as pd
from json import dumps
import logging

# Setup basic logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Provide the connection string to the Event Hubs namespace and the Event Hub name
connection_str = 'YOUR ENDPOINT'
eventhub_name = 'stock_market_real_time_eventhub'

# Load your data into a DataFrame
df = pd.read_csv('indexProcessed.csv')

# Create a producer client to send messages to the event hub
producer = EventHubProducerClient.from_connection_string(conn_str=connection_str, eventhub_name=eventhub_name)

# Infinite loop to continuously generate and publish events
while True:
  try:
      # Create a batch
      batch = producer.create_batch()

      # Randomly sample one record from the DataFrame and convert it to a dictionary
      dict_stock = df.sample(1).to_dict(orient='records')[0]
      print(dumps(dict_stock))

      # Serialize the dictionary to JSON and then create EventData
      event_data = EventData(dumps(dict_stock))
      batch.add(event_data)

      # Try to send the batch of events to the event hub
      try:
          producer.send_batch(batch)
          logging.info("Batch sent successfully.")
      except Exception as e:
          logging.error("Failed to send batch: {}".format(str(e)))

  except Exception as e:
      logging.error("Failed during the batch creation/sending process: {}".format(str(e)))

#  finally:
#      # Close the connection
#      producer.close()
#      logging.info("Event Hub producer closed.")